# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(["wordnet", "punkt", "stopwords"])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

from gensim.models import Word2Vec 
import gensim
import pickle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName", con=engine)
#Y = df.drop("message", axis=1)
Y = df.iloc[:,4:]
X = df["message"]
#There is a 2 that shouldnt be there, replacing it with the mode
Y["related"] = np.where(Y["related"]==2, 0,1)

In [19]:
Y = df.iloc[:,4:].sum()
cat_values = Y.values
cat_names = Y.index

Y_ratio = Y/len(df)
cat_ratio_values = Y_ratio.values
cat_ratio_names = Y_ratio.index

In [20]:
len(Y)

36

In [21]:
Y

related                   20282
request                    4474
offer                       118
aid_related               10860
medical_help               2084
medical_products           1313
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1672
food                       2923
shelter                    2314
clothing                    405
money                       604
missing_people              298
refugees                    875
death                      1194
other_aid                  3446
infrastructure_related     1705
transport                  1201
buildings                  1333
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7297
floods                     2155
storm                      2443
fire    

In [23]:
cat_ratio_values

array([ 0.77364968,  0.17065914,  0.00450107,  0.41425084,  0.07949344,
        0.05008392,  0.02761672,  0.01796613,  0.03280439,  0.        ,
        0.06377785,  0.1114968 ,  0.08826671,  0.01544858,  0.02303937,
        0.0113671 ,  0.03337656,  0.04554471,  0.13144644,  0.06503662,
        0.04581172,  0.05084681,  0.02029295,  0.006065  ,  0.01079493,
        0.00457736,  0.0117867 ,  0.04390449,  0.27834147,  0.08220171,
        0.09318737,  0.01075679,  0.0936451 ,  0.02021666,  0.05248703,
        0.19358407])

In [12]:
y_series = Y.sum()/len(Y)

In [13]:
y_series.index

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [14]:
y_series.values

array([ 0.99282881,  0.17065914,  0.00450107,  0.41425084,  0.07949344,
        0.05008392,  0.02761672,  0.01796613,  0.03280439,  0.        ,
        0.06377785,  0.1114968 ,  0.08826671,  0.01544858,  0.02303937,
        0.0113671 ,  0.03337656,  0.04554471,  0.13144644,  0.06503662,
        0.04581172,  0.05084681,  0.02029295,  0.006065  ,  0.01079493,
        0.00457736,  0.0117867 ,  0.04390449,  0.27834147,  0.08220171,
        0.09318737,  0.01075679,  0.0936451 ,  0.02021666,  0.05248703,
        0.19358407])

Is there any variable that is constant? Yes, "child_alone"

In [3]:
variables = (Y.describe()[Y.describe().index == "std"] == 0).any()
variables[variables.values == True].index.values[0]

'child_alone'

In [4]:
Y.drop("child_alone", axis=1, inplace=True)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    #Normalize, Remove punctuation characters and lower text
    text = text.lower()
    text = re.sub(r"[^a-zA-Z]", " ", text)
    #Tokenize
    text = word_tokenize(text)
    #Remove stop words
    words = [w for w in text if w not in stopwords.words("english")]
    # Lemmetize, reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmed

In [6]:
X[:2]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

In [7]:
X[:5].apply(tokenize)

0    [weather, update, cold, front, cuba, could, pa...
1                                          [hurricane]
2                             [looking, someone, name]
3    [un, report, leogane, destroyed, hospital, st,...
4    [say, west, side, haiti, rest, country, today,...
Name: message, dtype: object

In [8]:
len(X)

26216

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
    ])

In [14]:
pipeline4 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
    ])

In [15]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),#, max_features=10)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [9]:
pipeline5 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, min_samples_split=4)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [17]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [18]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [19]:
pipeline4.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [11]:
pipeline5.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [20]:
y_test_pred = pipeline.predict(X_test)
target_names = y_test.columns.values#tolist()

In [21]:
print(classification_report(y_test.values, y_test_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       1.00      0.99      0.99      6507
               request       0.80      0.43      0.56      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.73      0.61      0.67      2670
          medical_help       0.60      0.10      0.17       535
      medical_products       0.71      0.06      0.11       344
     search_and_rescue       0.55      0.04      0.07       159
              security       0.33      0.01      0.02       116
              military       0.58      0.10      0.16       200
                 water       0.82      0.29      0.43       418
                  food       0.85      0.48      0.62       745
               shelter       0.80      0.27      0.41       581
              clothing       0.69      0.09      0.16        98
                 money       0.64      0.07      0.12       133
        missing_people       0.88      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
y_test_pred2 = pipeline2.predict(X_test)
target_names = y_test.columns.values#tolist()

In [23]:
print(classification_report(y_test.values, y_test_pred2, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6507
               request       0.85      0.50      0.63      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.69      0.71      2670
          medical_help       0.66      0.08      0.15       535
      medical_products       0.81      0.08      0.14       344
     search_and_rescue       0.82      0.06      0.11       159
              security       0.33      0.01      0.02       116
              military       0.61      0.06      0.10       200
                 water       0.87      0.36      0.51       418
                  food       0.84      0.62      0.71       745
               shelter       0.80      0.40      0.53       581
              clothing       0.82      0.09      0.17        98
                 money       0.80      0.06      0.11       133
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
y_test_pred4 = pipeline4.predict(X_test)
target_names = y_test.columns.values#tolist()

In [25]:
print(classification_report(y_test.values, y_test_pred4, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6507
               request       0.84      0.48      0.61      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.77      0.65      0.70      2670
          medical_help       0.69      0.08      0.15       535
      medical_products       0.89      0.10      0.17       344
     search_and_rescue       0.80      0.08      0.14       159
              security       0.33      0.01      0.02       116
              military       0.60      0.04      0.08       200
                 water       0.85      0.41      0.55       418
                  food       0.87      0.54      0.66       745
               shelter       0.83      0.31      0.45       581
              clothing       0.73      0.08      0.15        98
                 money       0.88      0.05      0.10       133
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
y_test_pred5 = pipeline5.predict(X_test)
target_names = y_test.columns.values#tolist()

In [29]:
print(classification_report(y_test.values, y_test_pred5, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6507
               request       0.83      0.51      0.63      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.71      0.72      2670
          medical_help       0.66      0.10      0.17       535
      medical_products       0.78      0.08      0.15       344
     search_and_rescue       0.78      0.04      0.08       159
              security       0.00      0.00      0.00       116
              military       0.60      0.09      0.16       200
                 water       0.89      0.41      0.56       418
                  food       0.85      0.63      0.72       745
               shelter       0.82      0.40      0.54       581
              clothing       0.86      0.06      0.11        98
                 money       0.80      0.06      0.11       133
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [42]:
#for index, col in enumerate(Y.columns):
#    print(f"Feature: {col}")
#    print(classification_report(y_test[col], y_test_pred[:,index]))#, target_names=col))

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [32]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'tfidf__use_idf': (True, False),
        #'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2, 4]
            }
        

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), total= 2.0min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), total= 2.0min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), total= 2.0min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), total= 3.0min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), total= 3.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), total= 3.0min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 82.4min finished


In [33]:
cv.best_params_

{'clf__estimator__min_samples_split': 4,
 'tfidf__use_idf': False,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6507
               request       0.81      0.46      0.59      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.65      0.69      2670
          medical_help       0.60      0.06      0.11       535
      medical_products       0.86      0.12      0.22       344
     search_and_rescue       0.81      0.08      0.15       159
              security       0.50      0.01      0.02       116
              military       0.52      0.08      0.14       200
                 water       0.87      0.35      0.50       418
                  food       0.83      0.60      0.69       745
               shelter       0.79      0.37      0.50       581
              clothing       0.78      0.18      0.30        98
                 money       0.73      0.08      0.15       133
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [33]:
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

In [ ]:
class DenseTransformer(TransformerMixin, BaseEstimator):
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [28]:
pipeline3 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),#, max_features=20)),
        ('tfidf', TfidfTransformer()),
        #('to_dense', DenseTransformer()),
        #('clf', MultiOutputClassifier(GaussianNB()))
        ('clf', MultiOutputClassifier(SVC()))
        
    ])

Grid search takes extremely long time. Not feasible to run sadly.

In [ ]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'tfidf__use_idf': (True, False),
            }
        
cv = GridSearchCV(pipeline3, param_grid=parameters, verbose=2)

In [ ]:
#cv.fit(X_train[:50], y_train[:50])
cv.fit(X_train, y_train)

In [44]:
#y_pred = cv.predict(X_test)

In [45]:
#print(classification_report(y_test.values, y_pred, target_names=Y.columns))

In [29]:
pipeline3.fit(X_train, y_train)
y_pred = pipeline3.predict(X_test)

In [30]:
print(classification_report(y_test.values, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.99      1.00      1.00      6507
               request       0.00      0.00      0.00      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.00      0.00      0.00      2670
          medical_help       0.00      0.00      0.00       535
      medical_products       0.00      0.00      0.00       344
     search_and_rescue       0.00      0.00      0.00       159
              security       0.00      0.00      0.00       116
              military       0.00      0.00      0.00       200
                 water       0.00      0.00      0.00       418
                  food       0.00      0.00      0.00       745
               shelter       0.00      0.00      0.00       581
              clothing       0.00      0.00      0.00        98
                 money       0.00      0.00      0.00       133
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [12]:

pickle.dump(pipeline5, open('final_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.